In [18]:
 """Refresh patient cohort list.

This template script to generate the project cohort list. Some customizations may be needed to make it work for each project. 

Usage:
 
Add instructions here. 
"""

from omop_etl.utils import timeitc
from omop_etl.datastore import DataStore 
from omop_etl.bo import bo_query, format_bo_sql

store = DataStore('z:/test/config.yml')
BO_DOCNAME = store.config.bo_docname_cohort
END_DATE = store.config.end_date
DATABASE = 'DWS_OMOP' # store.config.project_database
SCHEMA = 'cohort'
COHORT_TABLE = 'PersonList'

# Use this list to add things to replace (placeholders) in the original BO query.
# Add pairs as (old_value, new_value)
replace_placeholders = [
    ('12/31/1900 00:0:0', END_DATE)
]

def load_cohort_query(dp_names, cohort_table, schema):
    """Return query string to load cohort into schema.cohort_table."""
    if hasattr(dp_names, '__iter__'):
        union_str = 'union '.join([f'select patnt_key from {SCHEMA}.{t} ' for t in dp_names])
    elif isinstance(dp_names, str):
        union_str = f'select patnt_key from {SCHEMA}.{dp_names}'
    else:
        raise TypeError

    return f"""
        INSERT INTO {schema}.{cohort_table} WITH (TABLOCK)
        SELECT DISTINCT *
        FROM (
            {union_str}
        ) x
    """

def refresh_cohort(stage=True, replace_placeholders=None):
    """[summary]

    Returns:
        [type]: [description]
    """
    store.truncate(SCHEMA, COHORT_TABLE)

    with store.bo_engine.connect() as con:
        bo_q = bo_query(BO_DOCNAME, con)
    
    # Stage all cohort tables
    if stage:
        for t in bo_q.keys():
            q = bo_q[t]
            sqlstring = format_bo_sql(q, t, DATABASE)

            # Replace placeholders
            if replace_placeholders: 
                for item in replace_placeholders:
                    old, new = item
                    sqlstring = sqlstring.replace(old, new)

            print(f'Staging {t} ...')
            print(store.execute(f"EXECUTE ('USE DWS_PROD;\n {sqlstring}')"))

    print(f'Loading cohort into {COHORT_TABLE} ...')
    cohort_q = load_cohort_query(bo_q.keys(), COHORT_TABLE, SCHEMA)
    
    return store.execute(cohort_q)


In [19]:
# refresh_cohort(replace_placeholders=replace_placeholders)
with store.bo_engine.connect() as con:
    bo_q = bo_query(BO_DOCNAME, con)

In [20]:
for t in bo_q.keys():
    q = bo_q[t]
    sqlstring = format_bo_sql(q, t, DATABASE)

In [22]:
bo_q['ICD_CodingDetail']

"SELECT  ALL_PATIENTS.PATNT_KEY,  ALL_PATIENTS.PATNT_ID,  ALL_PATIENT_IDENTITIES.IDENT_ID_INT,  JAX_MRN.IDENT_ID_INT FROM  ALL_PATIENTS RIGHT OUTER JOIN ALL_PATIENT_SNAPSHOTS ON (ALL_PATIENT_SNAPSHOTS.PATNT_KEY=ALL_PATIENTS.PATNT_KEY)  RIGHT OUTER JOIN DIAGNOSIS_EVENT_DTL ON (DIAGNOSIS_EVENT_DTL.PATNT_SNAPSHT_KEY=ALL_PATIENT_SNAPSHOTS.PATNT_SNAPSHT_KEY)  LEFT OUTER JOIN SOURCE_DIAG_ICD_XREF ON (DIAGNOSIS_EVENT_DTL.ICD_TYPE = SOURCE_DIAG_ICD_XREF.ICD_TYPE AND DIAGNOSIS_EVENT_DTL.SOURCE_DIAG_CD_KEY = SOURCE_DIAG_ICD_XREF.SOURCE_DIAG_CD_KEY)  LEFT OUTER JOIN ALL_ICD_DIAGNOSIS_CODES ON (SOURCE_DIAG_ICD_XREF.DIAG_CD_KEY=ALL_ICD_DIAGNOSIS_CODES.DIAG_CD_KEY)  LEFT OUTER JOIN PATIENT_ENCOUNTER_DTL ON (DIAGNOSIS_EVENT_DTL.PATNT_ENCNTR_KEY=PATIENT_ENCOUNTER_DTL.PATNT_ENCNTR_KEY)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES ON (ALL_PATIENT_SNAPSHOTS.PATNT_KEY=ALL_PATIENT_IDENTITIES.PATNT_KEY and ALL_PATIENT_IDENTITIES.LOOKUP_IND = 'Y' and ALL_PATIENT_IDENTITIES.IDENT_ID_TYPE = 101)  LEFT OUTER JOIN AL

In [14]:
with timeitc('Refreshing cohort table '):
    refresh_cohort(replace_placeholders=replace_placeholders)

Staging Diagnosis ...


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near the keyword 'INTO'. (156) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near 'ALL_ICD_DIAGNOSIS_CODES'. (102)")
[SQL: EXECUTE ('USE DWS_PROD;
 DROP TABLE IF EXISTS DWS_OMOP.cohort.Diagnosis; SELECT DISTINCT   INTO DWS_OMOP.cohort.Diagnosis  FROM  ALL_PATIENTS RIGHT OUTER JOIN ALL_PATIENT_SNAPSHOTS ON (ALL_PATIENT_SNAPSHOTS.PATNT_KEY=ALL_PATIENTS.PATNT_KEY)  RIGHT OUTER JOIN DIAGNOSIS_EVENT_DTL ON (DIAGNOSIS_EVENT_DTL.PATNT_SNAPSHT_KEY=ALL_PATIENT_SNAPSHOTS.PATNT_SNAPSHT_KEY)  LEFT OUTER JOIN SOURCE_DIAG_ICD_XREF ON (DIAGNOSIS_EVENT_DTL.ICD_TYPE = SOURCE_DIAG_ICD_XREF.ICD_TYPE AND DIAGNOSIS_EVENT_DTL.SOURCE_DIAG_CD_KEY = SOURCE_DIAG_ICD_XREF.SOURCE_DIAG_CD_KEY)  LEFT OUTER JOIN ALL_ICD_DIAGNOSIS_CODES ON (SOURCE_DIAG_ICD_XREF.DIAG_CD_KEY=ALL_ICD_DIAGNOSIS_CODES.DIAG_CD_KEY)  LEFT OUTER JOIN PATIENT_ENCOUNTER_DTL O